# 结论 2023-12-07
和 dynamo export 一致

1. 不支持 Tensor 作为控制条件
2. 只支持静态控制场景

In [2]:
import torch

In [3]:
def fn(x):
    if x.sum() > 0:
        return torch.relu(x)
    return torch.sigmoid(x)


flag = torch.randn(1) > 0.5 # failed, Dynamic control flow is not supported at the moment
flag = torch.randn(1).item() > 0.5

def static_flag_fn(x):
    if flag:
        return torch.relu(x)
    return torch.sigmoid(x)

x = torch.randn(3)
out = torch.export.export(static_flag_fn, (x,))
print(out)

ExportedProgram:
    class GraphModule(torch.nn.Module):
        def forward(self, arg0_1: f32[3]):
            # 
            sigmoid: f32[3] = torch.ops.aten.sigmoid.default(arg0_1);  arg0_1 = None
            return (sigmoid,)
            
Graph Signature: ExportGraphSignature(parameters=[], buffers=[], user_inputs=['arg0_1'], user_outputs=['sigmoid'], inputs_to_parameters={}, inputs_to_buffers={}, buffers_to_mutate={}, backward_signature=None, assertion_dep_token=None)
Symbol to range: {}



In [4]:
torch.export.export(fn, (x, ))

UserError: Dynamic control flow is not supported at the moment. Please use functorch.experimental.control_flow.cond to explicitly capture the control flow

from user code:
   File "/tmp/ipykernel_149592/911394525.py", line 2, in fn
    if x.sum() > 0:

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information
